(solph-offset)=

# oemof-solph model with variable partload efficiency

The final modification in our energy system model is to implement the load dependent variable COP. To do that we can use
the `OffsetTransformer`. This component requires different input compared to our previous implementations. Instead of
a conversion factor connecting input with output as shown in eq. {eq}`simple-conversion`, we define a slope and an 
offset (eq. {eq}`slope-offset-conversion`). This also changes the overall efficiency value in an interesting way per eq.
{eq}`offset-efficiency`: We have nonlinear efficiency in a linear model. We can visualize the effect in
{numref}`heatpump-offset-transformer`. For that we load the TESPy results and plot the compressor power and COP over the
heat production.

```{math}
:label: simple-conversion
E_\text{out} = \eta \cdot E_\text{in}
```

```{math}
:label: slope-offset-conversion
E_\text{out} = E_0 + m \cdot \cdot E_\text{in}
```

```{math}
:label: offset-efficiency
\eta = \frac{E_\text{out}}{E_0 + m \cdot \cdot E_\text{in}}
```

```{glue:figure} heatpump-offset-transformer
:name: "heatpump-offset-transformer"

Compressor power and COP as function of the heat pump heat production at 7 °C.
```

In [ ]:
from utilities import load_tespy_coefficients, load_input_data
from matplotlib import pyplot as plt
import numpy as np


input_data = load_input_data().head(24*2).tail(24)
tespy_coefficients = load_tespy_coefficients()

example = tespy_coefficients.loc[7]

heat_production_range = np.linspace(0.5, 1) * 9.1e3
compressor_power = (heat_production_range - example.loc["offset"]) / example.loc["slope"]
cop = heat_production_range / compressor_power


fig, ax = plt.subplots(2, sharex=True)

ax[0].plot(heat_production_range, compressor_power)
ax[0].set_ylim([0, compressor_power.max() * 1.05])
ax[0].set_ylabel("Compressor power in W")

ax[1].plot(heat_production_range, cop)
ax[1].set_ylim([0, cop.max() * 1.05])
ax[1].set_ylabel("COP")

ax[1].set_xlabel("Heat production in W")
ax[1].set_xlim([0, heat_production_range.max() * 1.05])

plt.close()

In [ ]:
from myst_nb import glue
glue("heatpump-offset-transformer", fig, display=False)

We can transform the input data from the TESPy model by mapping them onto the ambient temperatures in a similar way as
we had done this for the linear model.

In [ ]:
input_data["slope"] = input_data["Ambient temperature (d°C)"].map(tespy_coefficients["slope"])
input_data["offset"] = input_data["Ambient temperature (d°C)"].map(tespy_coefficients["offset"])

Then we load the energy system and add the heat pump with the necessary changes:

In [ ]:
from utilities import create_energy_system_stub

es, bus_electricity, bus_heat_35C, thermal_storage, electricity_grid = create_energy_system_stub(input_data)

With respect to the previous version (solph-minimal-load), a the Transformer is replaced by an OffsetTransformer.

In [ ]:
import oemof.solph as solph

hp_thermal_power = 9.1  # kW

slope = input_data["slope"][:-1]
offset = input_data["offset"][:-1]/1e3
demand = input_data["Heat load (kW)"][:-1]


heat_pump = solph.components.OffsetTransformer(
        label=f"heat pump",
        inputs={bus_electricity: solph.Flow(
            nominal_value=5,
            min=0.0,
            nonconvex=solph.NonConvex(),
        )},
        outputs={bus_heat_35C: solph.Flow(
            nominal_value=hp_thermal_power,
            nonconvex=solph.NonConvex(),
            min=0.5,
        )},
        coefficients=[offset, slope]
)

es.add(heat_pump)

We can solve our model and have a look at the results.

In [ ]:
model = solph.Model(energysystem=es)
model.solve()
results = solph.processing.results(model)

In [ ]:
heat_supply = results[(heat_pump, bus_heat_35C)]["sequences"]["flow"]
storage_content = results[(thermal_storage, None)]["sequences"]["storage_content"]

plt.plot(np.sort(heat_supply)[::-1], "r-", label="heat supply")
plt.plot(np.sort(demand)[::-1], "b--", label="heat demand")
plt.ylabel("Power (kW)")
plt.grid()
plt.legend()

plt.figure()
plt.plot(heat_supply, "r-", label="heat supply", drawstyle="steps-post")
plt.plot(demand, "b--", label="heat demand", drawstyle="steps-post")
plt.plot(storage_content, "k-", label="storage content")
plt.ylabel("Power (kW) or Energy (kWh)")
plt.grid()
plt.legend()

electricity_consumption = float(results[(electricity_grid, bus_electricity)]["sequences"].sum())
print(f"Electricity demand: {electricity_consumption:.1f} kWh")
